In [48]:
import numpy as np
import pandas as pd
import datetime

In [49]:
# 寫入大盤
stock_rate_table = pd.read_csv('economic/stockrate/^TWII.csv',encoding="cp1252")
stock_rate_table.columns =  ['日期','開市大盤','大盤最高','大盤最低','收市大盤','收市大盤經調整','成交量']

# 寫入貨幣匯率
exchange_table = pd.read_csv('economic/exchange/200610_202210.csv',encoding="cp1252")
exchange_table.columns = ['日期','美元／新台幣','人民幣／新台幣','歐元／美元','美元／日幣','英鎊／美元','澳幣／美元','美元／港幣','美元／人民幣','美元／南非幣','紐幣／美元']

# 將大盤和匯率合併為econimic_table
econimic_table = pd.merge(stock_rate_table,exchange_table, how='left',on='日期')

#載入cliwc解析結果，並將日期以文字格式另寫到其他column，以便後續合併
cliwc_parse_table = pd.read_csv('PttStock_CliwcParsed1008.csv')
cliwc_parse_table['日'] = cliwc_parse_table['日'].astype(str)
cliwc_parse_table['日'] = cliwc_parse_table['日'].replace(['1','2','3','4','5','6','7','8','9'],['01','02','03','04','05','06','07','08','09'])
cliwc_parse_table['月'] = cliwc_parse_table['月'].replace(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],['01','02','03','04','05','06','07','08','09','10','11','12'])
cliwc_parse_table['年'] = cliwc_parse_table['年'].astype(int)
cliwc_parse_table['日期']= cliwc_parse_table['日']+"/"+cliwc_parse_table['月'].astype(str)+"/"+cliwc_parse_table['年'].astype(str)
#cliwc另將日期轉換為星期寫到其他column，同時將月、日轉為整數型式，以便後續分析。
cliwc_parse_table['星期'] = pd.to_datetime(cliwc_parse_table['日期'], format='%d/%m/%Y').dt.dayofweek
cliwc_parse_table = cliwc_parse_table.drop(['作者', '標題','時'], axis=1)
cliwc_parse_table['月'] = cliwc_parse_table['月'].astype(int)
cliwc_parse_table['日'] = cliwc_parse_table['日'].astype(int)


###將cliwc解析結果與econimic_table合併為content_table###

#將cliwc以日期分組，以把cliwc每天文章與留言的content分數加總
content_sum = cliwc_parse_table[cliwc_parse_table.columns[3:-1]].groupby('日期').sum()
#將cliwc日期相關的column以日期分組，讓各項資料不重複
content_week = cliwc_parse_table.groupby('日期').mean()[['星期']]
content_year = cliwc_parse_table.groupby('日期').mean()[['年']]
content_month = cliwc_parse_table.groupby('日期').mean()[['月']]
content_date = cliwc_parse_table.groupby('日期').mean()[['日']]
#合併時間相關資料表與cliwc資料
content = content_sum.join(content_week, on='日期')
content = content.join(content_year, on='日期')
content = content.join(content_month, on='日期')
content = content.join(content_date, on='日期')
#把econimic_table需要的column取出，並將當天收市大盤column向前平移，以讓隔天的收市大盤放到當天
e = econimic_table[['日期', '美元／新台幣', '收市大盤']]
e = e.fillna(method = 'ffill')
e['隔日收市大盤'] = list(e['收市大盤'])[1:]+[list(e['收市大盤'])[-1]] #13409為最後一筆資料，當天的大盤指數
e['前日收市大盤'] = [list(e['收市大盤'])[0]]+list(e['收市大盤'])[0:-1] #13409為最後一筆資料，當天的大盤指數

#把content和econimic_table以日期合併(讓econimic_table多餘的資料被移除)
content = pd.merge(content,e, how='left', on='日期')
content.index = content['日期']
content = content.iloc[:,1:]
content.index = pd.to_datetime(content.index, format='%d/%m/%Y')
content = content.sort_index()


###分析資料整理###

#匯率在假日時會沒有資料，所以以前面最後一天有資料填補
content['美元／新台幣'] = content['美元／新台幣'].fillna(method = 'ffill')
#大盤在假日時會沒有資料，所以以前面最後一天有資料填補
content['收市大盤'] = content['收市大盤'].fillna(method = 'ffill')
content['隔日收市大盤'] = content['隔日收市大盤'].fillna(method = 'ffill')
content['前日收市大盤'] = content['前日收市大盤'].fillna(method = 'ffill') ##
#計算出隔日收市大盤的漲跌，作為後續預測的label
content['今日收市大盤漲跌'] = content['收市大盤']  - content['前日收市大盤']
content['隔日收市大盤漲跌'] = content['隔日收市大盤']  -content['收市大盤'] ##
#最後，因為週五及週六，隔天都沒有開市，可能會造成分析誤差，所以將週五週六的資料移除。(以週日預測週一)
content = content[content['星期']!=4]
content = content[content['星期']!=5]
content = content.drop(['隔日收市大盤', '前日收市大盤'], axis=1)

#將資料分散，以隨機分布，取1/10作為valid用，9/10作為train用
content = content.sample(frac=1).reset_index(drop=True)
content_valid = content.iloc[:(content.shape[0])//10,:]
content_train = content.iloc[(content.shape[0])//10:,:]

#寫出data
content.to_csv('data1009test.csv', encoding='UTF-8-Sig')
# content_valid.to_csv('data1009valid.csv', encoding='UTF-8-Sig')
# content_train.to_csv('data1009train.csv', encoding='UTF-8-Sig')

In [50]:
content_train

,推數,噓數,→數,content_function,content_pronoun,content_ppron,content_i,content_we,content_you,content_shehe,...,message_nonflu,message_filler,星期,年,月,日,美元／新台幣,收市大盤,今日收市大盤漲跌,隔日收市大盤漲跌
0,4814,602,3426,10981,368,170,59,17,44,43,...,1225,1596,1.0,2022.0,10.0,4.0,31.748,13576.51953,276.03906,224.91016
1,6249,1013,4647,21200,631,272,104,20,42,91,...,1746,2167,3.0,2022.0,9.0,29.0,31.847,13534.25977,68.18946,-109.67969
2,5204,764,3498,12506,331,136,39,13,24,46,...,1303,1514,2.0,2022.0,10.0,5.0,31.568,13801.42969,224.91016,90.62012
3,5802,848,4323,19258,699,254,97,15,39,90,...,1582,1898,0.0,2022.0,10.0,3.0,31.868,13300.48047,-124.09961,276.03906
4,633,100,600,2876,99,48,22,2,16,7,...,267,379,1.0,2022.0,9.0,27.0,31.780,13826.58984,48.39941,-360.51953
5,2702,497,2523,14168,465,186,63,9,35,74,...,934,1389,6.0,2022.0,10.0,2.0,31.743,13424.58008,-109.67969,-124.09961
6,4883,731,3519,13461,344,126,41,8,12,55,...,1273,1587,3.0,2022.0,10.0,6.0,31.530,13892.04981,90.62012,-189.76954
7,6789,1162,5615,27604,1131,608,281,57,133,113,...,2091,2550,2.0,2022.0,9.0,28.0,31.871,13466.07031,-360.51953,68.18946


In [55]:
test = pd.DataFrame({'a':[1,2,3,4,5,6,7,8,9,10],'b':[1,3,5,7,9,11,13,15,17,19]})
test1 = test.copy()
for i in range(test.shape[0]):
    crit = test.iloc[i,1] + 6
    table = test[test.iloc[:,1]<crit]
    test1.iloc[i,0] = table.sum()[0] 

test1

,a,b
0,6,1
1,10,3
2,15,5
3,21,7
4,28,9
5,36,11
6,45,13
7,55,15
8,55,17
9,55,19
